# Machine Learning

# Tutorial 2: Linear and polynomial regression

In this tutorial, we will cover:

- Linear regression, with linear and polynomial features
- Loss landscapes, L2 (Tikhonov) regularization
- The scikit-learn library

Authors:

- Prof. Emanuele Rodolà
- Based in part on original material by Dr. Antonio Norelli and Dr. Luca Moschella

Course:

Lectures and notebooks at https://erodola.github.io/ML-s2-2024/

#### Instructions
We encourage you to form small groups of 2-3 people to read and discuss the notebooks together.

Run the code and play with it! It is very easy to edit the code locally and make small experiments. Try whatever comes to your mind, this is the best way to learn! Python notebooks are designed to be used in this way, that's why we chose them for the DLAI lab sessions.

There will be some exercises, try to do them by yourself, and when everyone in your group has finished, compare the solutions with each other.

When something is not clear or you have a question, raise your hand and we will come to you, or directly approach us.

Remember that parts denoted by the open book 📖 are _optional_.

Let's start!

### Import dependencies

In this notebook we will extensively use **Plotly**, a powerful plotting library (there are many [visualization libraries](https://pyviz.org/overviews/index.html) for Python).
Effectively communicating your findings through plots and visualizations is an essential part of any scientific or engineering project.
Plotly is a modern library that makes interactive plots very easy to produce, and is very popular also outside of the ML community.

[Here](https://plot.ly/python/) you can find its documentation.

Remember: Don't take our suggestions as a dogma. If you prefer other plotting libraries, use them! In fact, we will be using different ones in this notebook to give you a gist of what's out there.

In [2]:
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

### A note on reproducibility

In our ML experiments, we will frequently encounter _randomness_. For example, whenever we want to generate some quick-and-dirty data to test our algorithms on the fly, we will just create some random matrices and points. In addition, many ML algorithms actually _rely on randomness_ due to the several beneficial properties it brings along! **Random forests** and **stochastic gradient descent** are two prominent examples.

Here's an idea: for our tests, we want to ensure that we always generate "the same randomness", so that anyone who will use our code will be able to reproduce exactly our results.

We do this by providing a hand-picked, fixed **seed** to the random number generator (RNG). The specific value of the feed is not important: what matters is that it's constant.

In [3]:
# Reproducibility stuff
import random
np.random.seed(42)  # initialize NumPy's RNG
random.seed(0)      # initialize Python's built-in RNG

## Linear models


Today we'll work with **linear models**. Don't underestimate their power! Linear models are simple, but they are at the heart of several effective pipelines. They will provide solid baselines for our experiments at the very least, and are a useful tool to study more advanced topics in modern deep learning.

### The regression problem


#### The general definition
We observe a phenomenon $\mathcal{P}$ where a variable of interest $t$ depends on another variable $x$.

**Problem**: Given a set of $N$ paired observations $(x, t)$, $\mathbf{D}=\{(x_1, t_1),...,(x_N, t_N)\}$, and a new *input* instance $x_i$, find the corresponding *target* $t_i$.

#### Our workspace

In our workspace, the underlying law governing our phenomenon $\mathcal{P}$ links $t$ to a $x \in [0,1]$ through a $\sin$ function.
For our purposes, we are going to synthetically generate a set of observations --our *dataset*-- following this rule:

$$t=\sin(2\pi x) + 0.1\epsilon \;\;\;\;\;\;\;\;\; \epsilon \sim \mathcal{N}(0,1)$$

where $\sim \mathcal{N}(0,1)$ means "*sampled from a gaussian distribution with $\mu=0$ and $\sigma=1$*".

In this way, we are simulating the measurement process of any phenomenon: there is a global and general regularity (that we wish to discover) but all observations are plagued by uncertainty, here modeled with gaussian noise. Uncertainty usually comes from the finite precision of our measuring instrument, but may also be due to sources of variability we do not care about. Sometimes the problem we are tackling is intrinsically stochastic --such as measurements in quantum mechanics or idealized scenarios like the multi-armed bandit-- and we want to figure out just the non-stochastic part of the underlying rule.

This is our dataset:

In [4]:
x = np.random.rand(10)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(10)

In [ ]:
fig = px.scatter(x=x, y=t)
fig.update_layout(width=600, height=400)
fig.show()

It seems we have very little to work with! Just a handful of noisy points.

How is it possible to devise a rule $f$ that is valid in general --i.e. for all the *infinite* points $x_i$ between 0 and 1 shall occur $t_i=f(x_i)$-- from such a *finite* set of observations?

If we collect just 10 points and each of them agrees with a precise rule, why does that give us *any grounds* --even probabilistic grounds-- to expect the 11[$^{st}$](https://translate.google.com/translate?sl=it&tl=en&u=https://it.wikipedia.org/wiki/Tacchino_induttivista) point to follow the same rule?

This is the essence of the so called [*Problem of induction*](https://ar5iv.labs.arxiv.org/html/1108.1791#S7) raised by David Hume more than two centuries ago. Still today this problem is at the core of the philosophical debate on the nature of knowledge.

### Where to restart from?

We will take a very pragmatic point of view. Even if induction cannot be justified in general, we note that "often it seems to work". Therefore we proceed with our quest to find $f$.

First of all, we observe that the quality of our guess $f$ depends crucially on how much information about the phenomenon $\mathcal{P}$ we are given in advance, i.e. what **priors** do we have.



**EXERCISE**: Suppose you have to guess this general rule $f$ to make predictions for new $x_i \in [0,1]$. In which of the following scenarios would you like to find yourself to make the most accurate predictions?

Write down a ranking from best to worst scenario, and then compare your ranking with those of your neighbors.

For each scenario below, we give you a description of the available priors together with a plot.


> **Scenario A**
>
> *priors:*
> - 20 observations $(x_i,t_i)$ in the interval $[0,1]$ affected by a gaussian noise $0.1*\epsilon$ where $\epsilon \sim \mathcal{N}(0,1)$.

> **Scenario B**
>
> *priors:*
> - 20 observations $(x_i,t_i)$ in the interval $[0,1]$ affected by a gaussian noise $0.5*\epsilon$ where $\epsilon \sim \mathcal{N}(0,1)$

> **Scenario C**
>
> *priors:*
> - 40 observations $(x_i,t_i)$ in the interval $[0,1]$ affected by a gaussian noise $0.1*\epsilon$ where $\epsilon \sim \mathcal{N}(0,1)$

> **Scenario D**
>
> *priors:*
> - 100 observations $(x_i,t_i)$ in the interval $[0,0.3]$ affected by a gaussian noise $0.1*\epsilon$ where $\epsilon \sim \mathcal{N}(0,1)$

> **Scenario E**
>
> *priors:*
> - 10 observations $(x_i,t_i)$ in the interval $[0,1]$ affected by a gaussian noise $0.1*\epsilon$ where $\epsilon \sim \mathcal{N}(0,1)$
> - you know that $f$ is in the form $f(x)=\sin(kx)$ with $k\in \mathbb{R}$


In [5]:
# @title Scenario plots

xA = np.random.rand(20)
tA = np.sin(2 * np.pi * xA) + 0.1 * np.random.randn(20)

xB = np.random.rand(20)
tB = np.sin(2 * np.pi * xB) + 0.5 * np.random.randn(20)

xC = np.random.rand(40)
tC = np.sin(2 * np.pi * xC) + 0.1 * np.random.randn(40)

xD = np.random.rand(100) * 0.3
tD = np.sin(2 * np.pi * xD) + 0.1 * np.random.randn(100)

xE = np.random.rand(10)
tE = np.sin(2 * np.pi * xE) + 0.1 * np.random.randn(10)

fig = make_subplots(
    rows=1, cols=5,
    subplot_titles=("Scenario A", "Scenario B", "Scenario C", "Scenario D", "Scenario E"))

fig.add_trace(go.Scatter(x=xA, y=tA, mode='markers', marker=dict(color="mediumpurple")),
              row=1, col=1)

fig.add_trace(go.Scatter(x=xB, y=tB, mode='markers', marker=dict(color="mediumpurple")),
              row=1, col=2)

fig.add_trace(go.Scatter(x=xC, y=tC, mode='markers', marker=dict(color="mediumpurple")),
              row=1, col=3)

fig.add_trace(go.Scatter(x=xD, y=tD, mode='markers', marker=dict(color="mediumpurple")),
              row=1, col=4)

fig.add_trace(go.Scatter(x=xE, y=tE, mode='markers', marker=dict(color="crimson")),
              row=1, col=5)

fig.update_xaxes(title_text="x", range=[0, 1])
fig.update_yaxes(title_text="", range=[-1.7, 1.7])
fig.update_layout(showlegend=False)

fig.show()

If you want to learn more, [here](http://bit.ly/3BLUE1BROWN2TWY9iI) you can find a nice discussion about an analogous exercise:
> Three Amazon resellers offer a book at essentially the same price. These are their ratings:
>
> - 100% positive out of 10 reviews
> - 96% positive out of 50 reviews
> - 93% positive out of 200 reviews
>
> Which rating is better?

> **EXERCISE (optional)**: You probably realized that in the previous exercise you wanted to be in the scenario with the largest amount of relevant information about the phenomenon $\mathcal{P}$. Now a tricky question: the dataset of which scenario, A or B, contains more *absolute* information? (i.e. Shannon information)
>
>Answer [here](http://bit.ly/VeritasiumRandom3b0rPBe), or more directly [here](https://en.wikipedia.org/wiki/Quantities_of_information#Self-information).

### Being linear in the parameters does not mean linear in the input

In this tutorial we will restrict our search of $f$ among parametrized functions $f_\theta$ that depend *linearly* on their finite set of parameters $\theta = \{a,b,c,...\}$.

Possible choices of $f$ are
- $f_1(x) = ax + b$
- $f_2(x) = ax^2 + bx + c$
- $f_3(x) = a \sin(2 \pi x) + bx + c$

In [6]:
# @title Some random plots of these linear models


x_funcs = np.arange(0., 1.1, 2./51)
theta_funcs = np.random.randn(10, 3)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("f1", "f2", "f3"))

for a, b in theta_funcs[:,:2]:
    fig.add_trace(go.Scatter(x=x_funcs, y=a*x_funcs + b, mode='lines'),
                row=1, col=1)

for a, b, c in theta_funcs:
    fig.add_trace(go.Scatter(x=x_funcs, y=a*x_funcs**2 + b*x_funcs + c, mode='lines'),
                row=1, col=2)

for a, b, c in theta_funcs:
    fig.add_trace(go.Scatter(x=x_funcs, y=a*np.sin(2 * np.pi * x_funcs) + b*x_funcs + c, mode='lines'),
                row=1, col=3)

for i in range(3):
    fig.add_trace(go.Scatter(x=x, y=t, mode='markers', marker=dict(color="mediumpurple")),
                row=1, col=i + 1)

fig.update_xaxes(title_text="x", range = [0,1])
fig.update_yaxes(title_text="t", range = [-2,2])
fig.update_layout(showlegend=False)

fig.show()

### Loss landscapes

Let's focus now on the simple $f(x) = ax + b$.
Given our dataset $(\mathbf{x}, \mathbf{t})$ we will choose the parameters $a$ and $b$ that minimize a global error $E$, also known as *loss*.

The standard choice for $E$ in a regression problem is the Mean Squared Error, defined as:

$$E(\theta) = \frac{1}{N} \sum_{i=1}^N(f_\theta(x_i) - t_i)^2$$

Notice that apart from normalization factors, the MSE corresponds to the squared $L_2$ norm of the residual vector, namely $E=\|f_\theta(\mathbf{x}) - \mathbf{t}\|_2^2$ (here $f_\theta$ operates element-wise).

Let's now compute the MSE **loss landscape** for our dataset, so that we can visualize it. The landscape is simply defined as the value of the loss function computed over a subset of the parameter domain.





In [ ]:
resolution = 51

mse_losses = np.empty((resolution, resolution))
a_range = np.linspace(-5, 5, resolution)
b_range = np.linspace(-5, 5, resolution)

for i, a in enumerate(a_range):
    for j, b in enumerate(b_range):
        y_pred = a * x + b
        residuals = t - y_pred
        loss = np.einsum('i,i->', residuals, residuals)
        mse_losses[i][j] = loss

# let's ignore the division by N for this exercise

>**EXERCISE** Vectorize the piece of code above. i.e. the two `for` cycles should disappear.

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution

Y_pred = x[None, None, :] * a_range[:, None, None] + b_range[None, :, None] # let broadcasting do the job!
R = t[None, None, :] - Y_pred
L = np.sum(R**2, axis=2)
np.allclose(L, mse_losses)

In [ ]:
# @title MSE loss landscape plot
fig = go.Figure(data=[go.Surface(z=mse_losses, x=a_range, y=b_range)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(
    title="MSE loss landscape",
    scene = dict(
                xaxis_title='a',
                yaxis_title='b',
                zaxis_title='loss')
)


fig.show()

Look at the [convexity](https://en.wikipedia.org/wiki/Convex_function) of the MSE loss. Can you do the linear regression by sight hovering the mouse over it?

>**EXERCISE** Visualize the **loss landscape** of other loss choices, such as the $L_1$ norm of the residual vector ($\|f_\theta(\mathbf{x}) - \mathbf{x}\|_1)$ (absolute error) or the $L_{50}$ norm. Are these still convex?  

In [ ]:
# ✏️ your code here

In [ ]:
# @title 👀 Solution (see how we did the slider!) { run: "auto" }

p = 2  #@param {type:"slider", min:1, max:50, step:1}

predictions = a_range[:, None, None] * x[None, None, :]  + b_range[None, :, None]
residuals = np.abs(t[None, None, :] - predictions) ** p
losses = (residuals.sum(-1) ** (1/p)) / predictions.shape[-1]


fig = go.Figure(data=[go.Surface(z=losses, x=a_range, y=b_range)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(
    title=f"L{p} landscape",
    scene = dict(
                xaxis_title='a',
                yaxis_title='b',
                zaxis_title='loss')
)
fig.show()

### "Tear down this loss[!](https://youtu.be/IguMXrgfrg8)"

Now we will explicitily compute the parameters $\theta$ for several polynomial fits, defined in matrix notation as:

![polyfit matrix notation](https://drive.google.com/uc?export=view&id=1e4AsSC00ivIXW4c35VNLladTcCTY4dpL)

As seen in the lecture, we can analytically find the parameters $\theta$ that minimize the MSE loss by solving a linear system:

$$\theta = (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y}$$

Let's do it for a degree-3 polynomial fit.





In [ ]:
# Poly3 fit

X = np.ones((t.shape[0], 4), dtype=np.float64)  # double precision is needed for this calculation, especially with high degree polyfits
X[:,0] = x ** 3
X[:,1] = x ** 2
X[:,2] = x

>**EXERCISE**: complete the code

In [ ]:
# ✏️ your code here (look at the formula above!)
theta = None

In [ ]:
# @title 👀 Solution (without einsum)

theta = np.linalg.inv(X.transpose() @ X) @ X.transpose() @ t

In [ ]:
# @title 👀 Solution (with einsum)

temp_inverse = np.linalg.inv(np.einsum('ij,jk->ik', X.transpose(), X))  # or np.einsum('ji,jk->ik', X, X)
theta = np.einsum('ij,jk,k->i', temp_inverse, X.transpose(), t)

In [ ]:
# @title 👀 Solution (numerically stable)

# The equation for theta requires us to solve a linear system.
# Instead of explicitly computing a matrix inverse and then multiplying, a more
# efficient and stable way is to use np.linalg.solve(), which is designed to
# handle this task more appropriately.

theta = np.linalg.solve(X.transpose() @ X, X.transpose() @ t)

In [ ]:
#@title Plotting our degree-3 polynomial fit

# NOTE: if you get an error saying that x_funcs is undefined, execute the code
# cell "Some random plots of these linear models", earlier in this notebook.

a, b, c, d = theta  # unpack the ndarray
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=t, name='dataset', mode='markers', marker=dict(color="mediumpurple")))
fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), name='ground truth', mode='lines', line=dict(color="lightgreen")))
fig.add_trace(go.Scatter(x=x_funcs, y=a*x_funcs**3 + b*x_funcs**2 + c*x_funcs + d, name='poly3 fit', mode='lines', line=dict(color="lightsalmon")))
fig.update_xaxes(title_text="x", range = [0,1])
fig.update_yaxes(title_text="t", range = [-1.5,1.5])
fig.update_layout(width=600, height=400)
fig.show()

Let's now compare several polynomial fits with different degrees.

Play a bit with the degrees while taking a look at the coefficients $\theta$.

Which one is the best in *your judgement*?
Which one is the best according to the MSE loss?

An evergreen reminder that here fits well:
> **be skeptical!** Especially about the numbers that should tell us if the results we get are good or bad. Are these really evaluating what should be evaluated?

In [ ]:
# @title Polynomial fits (press play at least once to trigger the self-update) { run: "auto" }

degree = {}
deg_a = 1  #@param {type:"slider", min:0, max:15, step:1}
deg_b = 4  #@param {type:"slider", min:0, max:15, step:1}
deg_c = 7 #@param {type:"slider", min:0, max:15, step:1}

degree[0], degree[1], degree[2] = deg_a, deg_b, deg_c

max_degree = max(degree.values())
X_full = np.ones((t.shape[0], max_degree + 1), dtype=np.float64)  # double precision is needed for this calculation!
for i in range(max_degree):
    X_full[:,i] =  x ** (max_degree - i)

theta_collection = {}

for i in range(3):
    X = X_full[:,-(degree[i] + 1):]
    temp_inverse = np.linalg.inv(np.einsum('ij,jk->ik', X.transpose(), X))
    theta_collection[i] = np.einsum('ij,jk,k->i', temp_inverse, X.transpose(), t)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(f"poly{degree[0]} fit", f"poly{degree[1]} fit", f"poly{degree[2]} fit"))

x_funcs = np.arange(0,1,1./100)
y_funcs = {}
for i in range(3):
    theta_funcs = theta_collection[i]
    X_funcs = np.ones((x_funcs.shape[0], degree[i] + 1), dtype=np.float64)
    for j in range(degree[i]):
        X_funcs[:,j] =  x_funcs ** (degree[i] - j)
    y_funcs[i] = np.einsum('ij,j->i', X_funcs, theta_funcs)
    coefficients_string = '<br><br><br>' + '<br>'.join([c + f': {np.round(theta_funcs[i], decimals=2)}' for i, c in enumerate('abcdefghijklmnopqrstuvwxyz'[:len(theta_funcs)])])

    fig.add_trace(go.Scatter(x=x, y=t, mode='markers', marker=dict(color="mediumpurple")), row=1, col=i + 1)
    fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), mode='lines', line=dict(color="lightgreen")), row=1, col=i + 1)
    fig.add_trace(go.Scatter(x=x_funcs, y=y_funcs[i], mode='lines', line=dict(color="lightsalmon")), row=1, col=i + 1)
    fig.update_xaxes(title_text="x" + coefficients_string, range = [0,1], row=1, col=i+1)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="t", range = [-5,5])
fig.update_layout(height=700, width=1300, title_text="Polynomial fits and their coefficients")
fig.show()

Nevertheless you should find *evidence* to support your judgement. Our intuition right now is that "High-degree polynomial fits lead to a high generalization error for this dataset", how can we test it?
As you have learnt in the lecture, a useful study you can perform consists in evaluating the MSE loss on a *test set*, i.e., on samples not used for training.

Such as the ones that are going to magically appear in the next cell.

In [ ]:
x_test = np.random.rand(10)
t_test = np.sin(2 * np.pi * x_test) + 0.1 * np.random.randn(10)

> **EXERCISE**: Do a nice plot entitled "MSE loss **vs** degree of the polynomial fit" that shows the trends of the training set and the test set.

In [ ]:
# ✏️ your code here

**NOTE**

Typically, when you are designing a learning model, you would get all the available data at the beginning. Then, you split this data into training, validation, and test sets.

To recap:

* **Training set**: the data we use to estimate our model's parameters.
* **Validation set**: new data we use to evaluate our model, and possibly tune its hyperparameters.
* **Test set**: new data we use to evaluate the final model.

### Data leakage

> ⚠️ Using the test set to tune your model's design and hyperparameters is not just cheating: it is *conceptually wrong*.
>
> In the real world, you almost never have access to the test set. For example, ChatGPT has never seen the crazy prompts we give it, but it still works well. Our input forms ChatGPT's test set. We can conclude that ChatGPT generalizes well to unseen data.

### Regularization

Still, it is not always desirable to work with simple models with few parameters.

Think about the true link between the input variable $x$ and the target variable $t$, it is through a $\sin$ function.
And a $\sin$ function written in a polynomial basis (i.e. [its Taylor series](https://www.wolframalpha.com/input?i=taylor+series+sin+x)) contains terms of *every* odd degree, i.e. infinite terms. Our approximation of the ground truth function should benefit from high order terms, since they are *needed* to express a $\sin$ function.

We want to have many parameters, and we want them to do the right thing: to not overfit.
This is why we need **regularizers**.

Let's start with the simplest and most universal regularizer, *more data*.


In [ ]:
# @title Regularized polynomial fits: more data (again: play at least once) { run: "auto" }

training_dataset_size = 75  #@param {type:"slider", min:1, max:100, step:1}
temp_x, temp_t = x, t
x = np.random.rand(training_dataset_size)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(training_dataset_size)

deg_1 = 2  #@param {type:"slider", min:0, max:15, step:1}
deg_2 = 6  #@param {type:"slider", min:0, max:15, step:1}
deg_3 = 13  #@param {type:"slider", min:0, max:15, step:1}

degree = {}
degree[0], degree[1], degree[2] = deg_1, deg_2, deg_3

max_degree = max(degree.values())
X_full = np.ones((t.shape[0], max_degree + 1), dtype=np.float64)  # double precision is needed for this calculation!
for i in range(max_degree):
    X_full[:,i] =  x ** (max_degree - i)

theta_collection = {}

for i in range(3):
    X = X_full[:,-(degree[i] + 1):]
    temp_inverse = np.linalg.inv(np.einsum('ij,jk->ik', X.transpose(), X))
    theta_collection[i] = np.einsum('ij,jk,k->i', temp_inverse, X.transpose(), t)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(f"poly{degree[0]} fit", f"poly{degree[1]} fit", f"poly{degree[2]} fit"))

x_funcs = np.arange(0,1,1./100)
y_funcs = {}
for i in range(3):
    theta_funcs = theta_collection[i]
    X_funcs = np.ones((x_funcs.shape[0], degree[i] + 1), dtype=np.float64)
    for j in range(degree[i]):
        X_funcs[:,j] =  x_funcs ** (degree[i] - j)
    y_funcs[i] = np.einsum('ij,j->i', X_funcs, theta_funcs)
    coefficients_string = '<br><br><br>' + '<br>'.join([c + f': {np.round(theta_funcs[i], decimals=2)}' for i, c in enumerate('abcdefghijklmnopqrstuvwxyz'[:len(theta_funcs)])])

    fig.add_trace(go.Scatter(x=x, y=t, mode='markers', marker=dict(color="mediumpurple")), row=1, col=i + 1)
    fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), mode='lines', line=dict(color="lightgreen")), row=1, col=i + 1)
    fig.add_trace(go.Scatter(x=x_funcs, y=y_funcs[i], mode='lines', line=dict(color="lightsalmon")), row=1, col=i + 1)
    fig.update_xaxes(title_text="x" + coefficients_string, range = [0,1], row=1, col=i+1)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="t", range = [-5,5])
fig.update_layout(height=700, width=1300, title_text="Polynomial fits and their coefficients")
fig.show()

x, t = temp_x, temp_t

Unfortunately, we do not always have access to more data. Usually our goal is precisely to learn a general rule from a *few* samples, take for instance the [ARC challenge](https://www.kaggle.com/c/abstraction-and-reasoning-challenge/overview).

Indeed intelligence seems more related to the (small) ratio between the amount of available data and the prediction performance, rather than just to the prediction performance.

Have you noticed the huge values of $\theta$ in the case of overfitting? Aren't the coefficients of the high-order terms in the Taylor expansion of $\sin (x)$ supposed to be smaller and smaller?
What if we try to discourage this overfitting behavior by adding to the loss a penalization term proportional to the norm of $\theta$?

You have already seen (or will soon see) this very common regularization technique in the theoretical lectures:
$$E'(\theta) = \lambda \|\theta\|^2 + \frac{1}{N} \sum_{i=1}^N(f_\theta(x_i) - t_i)^2$$

> 📖 **EXERCISE**: Obtain the vectorized closed-form solution for $\theta$ that minimizes $E'$. (The one for $E$ was $\theta = (\mathbf{X}^\top \mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y}$)
>
> **Hint**: This is the first step:
> $$ E'(\theta) = (\mathbf{y} - \mathbf{X}\theta)^\top(\mathbf{y} - \mathbf{X}\theta) + \lambda\theta^\top\theta$$
>
> Solve the exercise using pen and paper, the good old way.

When you're done, let's play with the $\lambda$ parameter.

In [ ]:
# @title Regularized polynomial fits: weight decay { run: "auto" }

deg_1 = 2  #@param {type:"slider", min:0, max:15, step:1}
deg_2 = 6  #@param {type:"slider", min:0, max:15, step:1}
deg_3 = 15  #@param {type:"slider", min:0, max:15, step:1}
lam =   0.0001#@param {type:"number"}

degree = {}
degree[0], degree[1], degree[2] = deg_1, deg_2, deg_3

max_degree = max(degree.values())
X_full = np.ones((t.shape[0], max_degree + 1), dtype=np.float64)  # double precision is needed for this calculation!
for i in range(max_degree):
    X_full[:,i] =  x ** (max_degree - i)

theta_collection = {}

for i in range(3):
    X = X_full[:,-(degree[i] + 1):]
    temp_inverse = np.linalg.inv(np.einsum('ij,jk->ik', X.transpose(), X) + lam * np.eye(degree[i] + 1))
    theta_collection[i] = np.einsum('ij,jk,k->i', temp_inverse, X.transpose(), t)

fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=(f"poly{degree[0]} fit", f"poly{degree[1]} fit", f"poly{degree[2]} fit"))

x_funcs = np.arange(0,1,1./100)
y_funcs = {}
for i in range(3):
    theta_funcs = theta_collection[i]
    X_funcs = np.ones((x_funcs.shape[0], degree[i] + 1), dtype=np.float64)
    for j in range(degree[i]):
        X_funcs[:,j] =  x_funcs ** (degree[i] - j)
    y_funcs[i] = np.einsum('ij,j->i', X_funcs, theta_funcs)
    coefficients_string = '<br><br><br>' + '<br>'.join([c + f': {np.round(theta_funcs[i], decimals=2)}' for i, c in enumerate('abcdefghijklmnopqrstuvwxyz'[:len(theta_funcs)])])

    fig.add_trace(go.Scatter(x=x, y=t, mode='markers', marker=dict(color="mediumpurple")), row=1, col=i + 1)
    fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), mode='lines', line=dict(color="lightgreen")), row=1, col=i + 1)
    fig.add_trace(go.Scatter(x=x_funcs, y=y_funcs[i], mode='lines', line=dict(color="lightsalmon")), row=1, col=i + 1)
    fig.update_xaxes(title_text="x" + coefficients_string, range = [0,1], row=1, col=i+1)

fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="t", range = [-5,5])
fig.update_layout(height=700, width=1300, title_text="Polynomial fits and their coefficients")
fig.show()

Not so easy to tune, isn't it?

Look at the magnitude of those parameters though, not spiking up anymore!

It's a good moment to take a break ☕🍩, a new library awaits us!

## scikit-learn

Scikit-learn (also known as "sklearn" for brevity) is a ML library for python. It implements several ML algorithms, including the ones we have been studying so far, making them especially easy to use and hiding all the pain. It's very convenient!

Importantly, scikit-learn **integrates well with matplotlib, plotly, NumPy**, and many other libraries. This makes it quite natural to include it in our workflow.

**Pro tip:** As usual, bookmark the [docs](https://scikit-learn.org/stable/user_guide.html)!

In [ ]:
# @title Initial imports

from sklearn import linear_model
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

import random
np.random.seed(42)  # sklearn uses numpy's rng
random.seed(0)

### Linear regression

We start by replicating one of our initial tests: a simple fit of a linear model to some random data:

$$\arg\min_\theta \| \mathbf{X}\theta - \mathbf{y}\|_2^2$$

Note that sklearn expects a 2D array $\mathbf{X}$ to store the feature data. So if our data is one-dimensional, we must still encode it as a 2D tensor with just one column.

In [ ]:
# generate some random data
N = 20
x = np.random.rand(N)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(N)

# fit a linear model
reg = linear_model.LinearRegression()
_ = reg.fit(x[:, None], t)  # we converted x to a 2d tensor

Done! Simply create a `LinearRegression` object, and call the `fit()` method over the data. Let's do some plots now.

In [ ]:
# explicitly compute y=a*x+b using the learned a and b
y_pred2 = reg.coef_ * x + reg.intercept_

# ...or directly use the predict() method to do it for you!
y_pred = reg.predict(x[:, None])

plt.scatter(x, t, color="mediumpurple")
plt.plot(x, y_pred, color="lightsalmon", linewidth=3)
plt.plot(x, y_pred2, color="black", linewidth=1)
plt.show()

### Polynomial regression

Time to move on to polynomial fitting!

As we have seen during the theory class, this is still a _linear regression_ problem, but uses a different data matrix. Scikit-learn makes this very explicit, providing functions to transform the data from linear to **polynomial features** of any degree.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# generate some random data
N = 20
x = np.random.rand(N)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(N)

# compute polynomial features
poly = PolynomialFeatures(degree=3)
X = poly.fit_transform(x[:, None])  # again, it expects a 2D tensor
X, X.shape

Observe the following:

- We started with $N$ points, stored as **1d features** $[x]$ in tensor `x`.
- We obtained $N$ **4d polynomial features** $[1, x, x^2, x^3]$, stored in `X`.
- The first dimension is always $x^0 =1$.

We can now use these features to train any linear model.

💡 Thinking about data _features_, as opposed to the raw representation given as input, opens up a lot of directions! For example, with deep learning we will actually _learn_ the best features that solve a task.

In [ ]:
# linear fit with polynomial features
reg = linear_model.LinearRegression()
_ = reg.fit(X, t)

a, b, c, d = reg.coef_
e = reg.intercept_

# plot
x_funcs = np.arange(0., 1.1, 2./51)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=t, mode='markers', name='data', marker=dict(color="mediumpurple")))
fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), mode='lines', name='ground-truth', line=dict(color="lightgreen")))
fig.add_trace(go.Scatter(x=x_funcs, y=b*x_funcs+c*(x_funcs**2)+d*(x_funcs**3)+e, mode='lines', name='poly3 fit', line=dict(color="lightsalmon")))
fig.update_layout(width=600, height=400)
fig.show()

#### Fitting a trajectory

Let's do something a bit more interesting.

So far we have been trying to fit functions $f:\mathbb{R}\to\mathbb{R}$, but we can be more general than this and consider high-dimensional mappings $f:\mathbb{R}^m \to \mathbb{R}^n$. For example, a parametric curve on the plane would be modeled as a function $f:\mathbb{R}\to\mathbb{R}^2$.

Here's an example of what I like to call the [Zool](https://www.mobygames.com/game/5776/zool/) curve:

In [ ]:
t = np.linspace(0, np.pi, 100)
p = np.array((2*np.cos(2*t), np.sin(4*t)))

plt.plot(p[0, :], p[1, :], linestyle='-', color='b')
plt.grid(True)
plt.show()

> **EXERCISE**
>
> Fit a polynomial to the data given below, which was generated from a 3D spiral defined as:
>
> $$f(t) = (\cos(t), \sin(t), 0.1t)\,,\quad\quad t \in [0, 5\pi]$$
>
> Choose a polynomial degree by yourself. What happens if it's too small? Or too large?
>
> Create a nice plot of your solution, showing the polynomial curve passing through the training points. You can use Plotly's `go.Scatter3d` for drawing in 3D.
>
> _Remember:_ The true $f(t)$ is **unknown**, so you can't use the formula above to find a solution. The only input to your training model is $N$ data points $(t_i, f(t_i))$.

In [ ]:
# training data
t = np.linspace(0, 5*np.pi, 40)
p = np.array((np.cos(t), np.sin(t), 0.1*t))

# ✏️ your solution here


In [ ]:
# @title 👀 Solution


# training data
t = np.linspace(0, 5*np.pi, 40)
p = np.array((np.cos(t), np.sin(t), 0.1*t))

# Make sure you understand the problem precisely.
# - You are given training data as pairs (t, (x,y,z)).
# - You want to express each coordinate x, y, z as a *polynomial in t*.
# - Each coordinate will get its own set of learned parameters.
# Once you got this down, the rest is straightforward!

degree = 10

poly = PolynomialFeatures(degree=degree)
T = poly.fit_transform(t[:, None])

reg = linear_model.LinearRegression()
_ = reg.fit(T, p.transpose())

# do the plots
ts = t[: ,None] ** np.arange(degree+1)
P = np.einsum('ij, kj -> ik', ts, reg.coef_) + reg.intercept_

fig = make_subplots(rows=1, cols=2, subplot_titles=("Training data", "Polynomial fit"), specs=[[{'type': 'scene'}, {'type': 'scene'}]])
fig.add_trace(go.Scatter3d(x=p[0,:], y=p[1,:], z=p[2,:], mode='markers', marker=dict(opacity=1, size=5, color=t, colorscale="viridis")), row=1, col=1)
fig.add_trace(go.Scatter3d(x=p[0,:], y=p[1,:], z=p[2,:], mode='markers', marker=dict(opacity=0.5, size=5, color=t, colorscale="viridis")), row=1, col=2)
fig.add_trace(go.Scatter3d(x=P[:,0], y=P[:,1], z=P[:,2], mode='lines', line=dict(color='red', width=3)), row=1, col=2)
fig.update_layout(title='Trajectory fitting', showlegend=False)
fig.show()

> **EXERCISE:**
>
> What is the total number of parameters for the polynomial regression model used above?

#### Piecewise-polynomial fitting 📖

Some of the most interesting problems ML arise when you deal with **noisy, corrupted, or incomplete** data. Imagine being able to upscale a low-quality video to 1080p resolution; or removing the blur from a shaky photo; or fill up noisy parts of a badly recorded song with clean audio data. ML to the rescue!

Linear regression is not really the tool of the trade to solve this kind of problems. But let's try anyway. We are going to consider a **gray-scale image**, seen as a function:

$$f :\mathbb{R}^2 \to \mathbb{R} $$

Suppose we are given the pixel values only for some of the points. We are going to use degree-3 polynomial regression to "fill up the holes".

> 📁 **NOTE:**
> Make sure you have the image "cat.png" in your Drive folder.
>
> You don't really need to read and understand all the code in the next cell. We are just running regression on pixel data, and plot the predictions on the entire pixel grid.

In [ ]:
from sklearn.pipeline import make_pipeline
from PIL import Image

image_path = './cat.png'
image = Image.open(image_path).convert('L')  # convert to grayscale
image_array = np.array(image)

fraction = 0.8  # Fraction of pixels to keep
degree = 3  # Polynomial degree

# Flatten the image array and create coordinates grid
y, x = np.indices(image_array.shape)
x_flat = x.ravel()
y_flat = y.ravel()
image_flat = image_array.ravel()

# Randomly select a fraction of pixels
total_pixels = x_flat.size
num_selected = int(total_pixels * fraction)
selected_indices = np.random.choice(total_pixels, num_selected, replace=False)

# Prepare data for polynomial regression
X_selected = np.stack((x_flat[selected_indices], y_flat[selected_indices]), axis=1)
y_selected = image_flat[selected_indices]

# Fit Ridge regression model on selected pixels
model = make_pipeline(PolynomialFeatures(degree), linear_model.LinearRegression())
model.fit(X_selected, y_selected)

# Predict for the whole image
X_all = np.stack((x_flat, y_flat), axis=1)
image_recovered_flat = model.predict(X_all)
image_recovered = image_recovered_flat.reshape(image_array.shape)

# Visualize the original, recovered images, and kept pixels
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
axes[0].imshow(image_array, cmap='gray')
axes[0].set_title('Original ground-truth image')
axes[0].axis('off')

# Show the kept pixels on a black background
kept_pixels = np.zeros_like(image_array)
kept_pixels.flat[selected_indices] = image_array.flat[selected_indices]
axes[1].imshow(kept_pixels, cmap='gray')
axes[1].set_title('Noisy pixels')
axes[1].axis('off')

axes[2].imshow(image_recovered, cmap='gray')
axes[2].set_title('Recovered image')
axes[2].axis('off')

plt.show()

Not working too well, right? The recovered image is super blurry and flat. Try using higher-order polynomials to at least overfit the input pixel data, and see how far you can get.

> ⚠️ **On the number of parameters**
>
> Careful here!
>
> We are in the setting where the function to learn is $f: \mathbb{R}^2 \to \mathbb{R}$. Therefore, for a degree-2 polynomial regression problem, we are trying to fit a polynomial such that:
>
> $f(x, y) = ax^2 + by^2 + cxy + d$
>
> for all points $(x,y) \in \mathbb{R}^2$. Notice the **cross-terms** $xy$. Importantly, the parameters $\{a,b,c,d\}$ are going to be **the same for all pixels**.

> **EXERCISE:**
>
> What is the total number of parameters for the polynomial regression model used above?

Let's try to improve upon our horrible result. Instead of fitting a polynomial to all the pixels at once, let's split the given pixels into $M$ **patches**. Then, let's solve $M$ distinct polynomial regression problems, one per patch, and then stitch together the results to recompose a full image. Ready?

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from PIL import Image

# Load image
image_path = './cat.png'
image = Image.open(image_path).convert('L')
image_array = np.array(image)

# Parameters
patch_size = 5  #  patch size in pixels
degree = 3
fractions = np.linspace(0.98, 0.04, 10)  # Fraction of pixels to keep in each patch

fig, axes = plt.subplots(len(fractions), 2, figsize=(12, 4*len(fractions)))

for idx, fraction in enumerate(fractions):

    # Initialize the arrays for the reconstructed image and kept pixels
    reconstructed = np.zeros_like(image_array)
    kept_pixels = np.zeros_like(image_array)

    # Function to process each patch with selected fraction of pixels
    def process_patch(x_patch, y_patch, patch, fraction):
        # Flatten patch coordinates and values
        X_patch = np.array([x_patch.flatten(), y_patch.flatten()]).T
        y_patch = patch.flatten()

        # Randomly select a fraction of pixels
        selected = np.random.choice(X_patch.shape[0], int(X_patch.shape[0] * fraction), replace=False)
        X_selected = X_patch[selected]
        y_selected = y_patch[selected]

        # Fit linear regression model on selected pixels
        model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
        model.fit(X_selected, y_selected)

        # Predict for the whole patch and mark kept pixels
        predictions = model.predict(X_patch).reshape(patch.shape)
        kept_patch = np.zeros_like(patch)
        kept_patch.flat[selected] = y_patch.flat[selected]
        return predictions, kept_patch

    # Process the image in patches
    num_rows, num_cols = image_array.shape
    for i in range(0, num_rows, patch_size):
        for j in range(0, num_cols, patch_size):
            row_end = min(i + patch_size, num_rows)
            col_end = min(j + patch_size, num_cols)
            patch = image_array[i:row_end, j:col_end]
            x_patch, y_patch = np.meshgrid(np.arange(j, col_end), np.arange(i, row_end))

            # Reconstruct the patch and get kept pixels
            reconstructed_patch, kept_patch = process_patch(x_patch, y_patch, patch, fraction)
            reconstructed[i:row_end, j:col_end] = reconstructed_patch
            kept_pixels[i:row_end, j:col_end] = kept_patch

    axes[idx][0].imshow(kept_pixels, cmap='gray')
    axes[idx][0].set_title(f'Input data ({fraction*100:.2f}%)')
    axes[idx][0].axis('off')

    axes[idx][1].imshow(reconstructed, cmap='gray')
    axes[idx][1].set_title('Reconstructed Image')
    axes[idx][1].axis('off')

plt.show()

A few observations are in order:

- Even for very extreme cases (**4%** of all the input pixels!) we get reasonable results.
- The **smooth** portions of the image (e.g. the background) are always reconstructed well.
- The portions with lots of detail (e.g. the cat fur and whiskers) are the most difficult to recover.

We are essentially modeling our image as a **piecewise-polynomial function**. This allows us to accurately model more complex functions, at the cost of more parameters and more processing. As we shall see throughout the course, composing together simple functions is a powerful idea!

> **EXERCISE:**
>
> What is the total number of parameters for the piecewise-polynomial regression model used above?

### Ridge regression (linear + Tikhonov)

Let us now consider the $L_2$ penalty we used earlier to **penalize large parameter values**. We are now looking at the regularized problem:

$$\arg\min_\theta \| \mathbf{X}\theta - \mathbf{y}\|_2^2 + \alpha \|\theta\|_2^2$$

where $\alpha>0$ controls the trade-off between _data fidelity_ and _regularization_.

> **EXERCISE**
>
> If you haven't already solved this in the "Regularization" section earlier in the notebook:
>
> 1. Derive the normal equation for ridge regression.
> 2. Show that it is equivalent to standard linear regression, plus a _diagonal_ update.
>
>_Hint:_ start from rewriting the loss as $(\mathbf{X}\theta - \mathbf{y})^\top(\mathbf{X}\theta - \mathbf{y}) + \alpha \theta^\top\theta$.

We know how to solve this problem: write the normal equations and solve the system. Or we can use sklearn's `Ridge` class to do it for us!

In [ ]:
# generate some random data
N = 25
x = np.random.rand(N)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(N)

# compute polynomial features of high degree
poly = PolynomialFeatures(degree=17)
X = poly.fit_transform(x[:, None])

# standard linear regression
reg_linear = linear_model.LinearRegression()
_ = reg_linear.fit(X, t)

# ridge regression
reg_ridge = linear_model.Ridge(alpha=0.001)
_ = reg_ridge.fit(X, t)

I hope you appreciate how clean the code above is! Let's see how it's actually performing.

In [ ]:
# @title Plot linear vs. ridge regression

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(f"poly{X.shape[1]-1} fit", f"regularized poly{X.shape[1]-1} fit"))

X_funcs = x_funcs[:, None] ** np.arange(X.shape[1])
Y_linear = np.einsum('ij, j -> i', X_funcs, reg_linear.coef_) + reg_linear.intercept_
Y_ridge = np.einsum('ij, j -> i', X_funcs, reg_ridge.coef_) + reg_ridge.intercept_

fig.add_trace(go.Scatter(x=x, y=t, mode='markers', marker=dict(color="mediumpurple")), row=1, col=1)
fig.add_trace(go.Scatter(x=x, y=t, mode='markers', marker=dict(color="mediumpurple")), row=1, col=2)
fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), mode='lines', line=dict(color="lightgreen")), row=1, col=1)
fig.add_trace(go.Scatter(x=x_funcs, y=np.sin(2*np.pi*x_funcs), mode='lines', line=dict(color="lightgreen")), row=1, col=2)
fig.add_trace(go.Scatter(x=x_funcs, y=Y_linear, mode='lines', line=dict(color="lightsalmon")), row=1, col=1)
fig.add_trace(go.Scatter(x=x_funcs, y=Y_ridge, mode='lines', line=dict(color="lightsalmon")), row=1, col=2)
fig.update_yaxes(title_text="t", range = [-2,2])
fig.update_xaxes(title_text="x", range = [0,1])
fig.update_layout(showlegend=False)
fig.show()

> **EXERCISE**
>
> Using the data from the previous exercise, create a plot showing how the values of the learned _weights_ change with increasing $\alpha$.
>
> Draw one curve per weight, all in the same plot. For example, for a degree-17 polynomial, the plot should contain 18 curves.

In [ ]:
# ✏️ your solution here

# generate some random data
N = 25
x = np.random.rand(N)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(N)

# use these values for alpha
n_alphas = 100
alphas = np.logspace(-10, -2, n_alphas)  # spaced evenly on a log scale from 1e-10 to 1e-2

# ...

In [ ]:
# @title 👀 Solution

# generate some random data
N = 25
x = np.random.rand(N)
t = np.sin(2 * np.pi * x) + 0.1 * np.random.randn(N)

# use these values for alpha
n_alphas = 100
alphas = np.logspace(-10, -2, n_alphas)  # spaced evenly on a log scale from 1e-10 to 1e-2

# compute polynomial features of high degree
poly = PolynomialFeatures(degree=17)
X = poly.fit_transform(x[:, None])

# this will store the weights for all alphas (not storing the bias)
weights = np.zeros((len(alphas), X.shape[1]), dtype=np.float32)

for i, alpha in enumerate(alphas):
  reg_ridge = linear_model.Ridge(alpha=alpha)
  _ = reg_ridge.fit(X, t)
  weights[i] = reg_ridge.coef_

print(weights.shape)

# plot
ax = plt.gca()
ax.plot(alphas, weights)
ax.set_xscale("log")
plt.xlabel("alpha")
plt.ylabel("weights")
plt.axis("tight")
plt.show()

Observe how the parameters get _smaller_ for large $\alpha$. This is expected, because giving more weight to the regularizer implies a stronger penalty on the parameters.

> **EXERCISE**
>
> Using the same data as the previous exercise, do standard linear regression instead of ridge regression.
>
> What's the difference in scale between the parameters of the two models?

### Cross-validation

Finally, let's implement a simple $n$-fold cross-validation scheme. Recall the recipe from the theory class:

1. Split the training data into $n$ subsets ("folds").
2. Select one fold and keep it; we'll use it as a validation set.
3. Train you model on the remaining $n-1$ folds and save the model parameters.
4. Go back to step 2.
5. Stop when you've trained $n$ models.

This idea of training a model on all data except for one fold is also called **leave-one-out** in ML jargon.


> **EXERCISE**
>
> Using Scikit-learn, implement 5-fold cross-validation to test a model that performs degree-6 polynomial regression on data of your choice.
>
> To test your model on unseen data, use the `predict()` call from the `LinearRegression` class.
>
> Create a figure showing the predictions of the 5 models.